In [8]:
from tavily import TavilyClient
import openai
import os
import json
import re

# API 키 설정
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 초기화
tavily = TavilyClient(api_key=TAVILY_API_KEY)
openai.api_key = OPENAI_API_KEY


def extract_company_name(title: str) -> str:
    # 쉼표(,) 또는 첫 공백을 기준으로 회사명만 잘라냄
    return title.split(",")[0].split(" ")[0].strip()


def competitor_analysis_agent(state):
    domain = state.domain or "헬스케어"

    # 1. Tavily로 검색
    query = f"{domain} 스타트업 경쟁사 및 차별성"
    result = tavily.search(query=query, search_depth="advanced")
    documents = result["results"][:3]

    # 2. 문서 내용 정리
    document_text = "\n\n".join([f"{doc['title']}\n{doc['content']}" for doc in documents])

    # 3. GPT로 경쟁우위 점수 요청
    system_prompt = (
        "당신은 스타트업 투자 전문가입니다. 아래 경쟁사 정보들을 바탕으로 다음을 평가하세요:\n"
        "- 경쟁사보다 해당 스타트업이 차별성이 얼마나 뚜렷한지 0~10점으로 평가\n"
        "- 해당 점수의 이유 설명\n"
        "출력은 다음 JSON 형식:\n"
        "{\n"
        '  "competitive_score": float,\n'
        '  "competitive_reasoning": str\n'
        "}"
    )
    user_prompt = f"도메인: {domain}\n\n경쟁사 관련 정보:\n{document_text}"

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.5,
    )

    # 4. JSON 파싱
    raw_output = response["choices"][0]["message"]["content"]
    json_str = re.search(r"\{.*\}", raw_output, re.DOTALL).group(0)
    parsed = json.loads(json_str)

    # 5. 상태 업데이트
    state.market_analysis = state.market_analysis or {}

    # 경쟁사 문서 저장 (회사명만 추출)
    state.market_analysis["competitor_documents"] = [
        {
            "title": extract_company_name(doc["title"]),
            "url": doc["url"],
            "content": doc["content"]
        }
        for doc in documents
    ]

    # 점수 및 이유 저장
    state.market_analysis["competitive_score"] = parsed["competitive_score"]
    state.market_analysis["competitive_reasoning"] = parsed["competitive_reasoning"]

    return state
